# SK RAG pattern implementation

Learning objectives:

- A simple SK RAG pattern implementation using volatile memory

## Setup

### Load required .NET packages and supporting constants, classes, etc.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.19.230804.2-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Sqlite, 0.18.230725.3-preview"

#r "nuget: Microsoft.Extensions.Logging.Console"
#r "nuget: dotenv.net"

using System;
using System.IO;
using System.Text;
using System.Text.Json;
using System.Text.Json.Serialization;
using System.ComponentModel;
using System.Net.Http;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors;
using Microsoft.SemanticKernel.Connectors.Memory.Sqlite;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Utils/ConsoleLogger.cs
#!import Models/Models.cs

const string MemoryCollectionName = "LearningsCollection";

### Read the API Key and endpoints from environment variables or the .env file

In [ ]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var adaDeploymentName = "ada";
Console.WriteLine($"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...");

### Get a kernel instance configured for text completions and embeddings

In [ ]:
// I'm using a RAM stored Vector DB, but I can switch providers like Azure Search, DuckDB, SQLite, etc.
var ramStore = new VolatileMemoryStore();
var sqliteStore = await SqliteMemoryStore.ConnectAsync("./vectors.sqlite");

var kernel = Kernel.Builder
            .WithLogger(ConsoleLogger.Log)
            .WithAzureChatCompletionService(deploymentName, endpoint, apiKey)
            .WithAzureTextEmbeddingGenerationService(adaDeploymentName, endpoint, apiKey)
            .WithMemoryStorage(ramStore)
            .Build();

var skRagFuncDef = "{{$query}}\n\nUsing only the following text:\n\"\"\"\n{{$input}}\n\"\"\"";

## Ingestion

### Read and deserialize the JSON learnings data file

In [ ]:
var jsonFileContents = File.ReadAllText("data/learnings.json");
var learnings = System.Text.Json.JsonSerializer.Deserialize<List<Learning>>(jsonFileContents);
learnings

### Chunk the learnings & recommendations

**Note:** This is a simple chunker. It chunks by splitting the document into paragraphs. A more realistic chunker would try to optimize the token size limit, chunking smartly (not in the a middle of a paragraph or sentence), etc.

In [ ]:
// Keep a list of chunks
var chunks = new List<Chunk>();

// For each learning process the chunks
foreach(var learning in learnings)
{
    // Break the learnings into paragraphs
    var paragraphs = learning.Content.Split("\n\n");
    
    // For each paragraph create a chunk
    for(var i=0;i<paragraphs.Length;i++)
    {
        // Add the chunk to the list
        chunks.Add(new Chunk(learning.Id+"-"+(i+1),paragraphs[i]));
    }
}

### Save memories for every chunk

In [ ]:
foreach(var chunk in chunks)
{    
    await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: chunk.Id, text: chunk.Text);
}

## Grounding

### Retrieve the memory based on a query

In [ ]:
var query = await InteractiveKernel.GetInputAsync("What is your query?");

IAsyncEnumerable<MemoryQueryResult> queryResults =
                kernel.Memory.SearchAsync(MemoryCollectionName, query, limit: 3, minRelevanceScore: 0.77);


### Find memories based on query, and collect the text in the memories to augment the prompt

In [ ]:
// Keep a list of the memories
StringBuilder promptData = new StringBuilder();

await foreach (MemoryQueryResult r in queryResults)
{
    promptData.Append(r.Metadata.Text+"\n\n");
}

// Final augmented text
var augmentedText = promptData.ToString();
Console.WriteLine($"User:\n{query}\n\nNearest results:\n{augmentedText}")

## Process Prompt & Completion

### Create a SK function to process the prompt<br/>and execute the function

In [ ]:
const string ragFunctionDefinition = "{{$input}}\n\nText:\n\"\"\"{{$data}}\n\"\"\"";
var ragFunction = kernel.CreateSemanticFunction(ragFunctionDefinition, maxTokens: 1000);

var result = await kernel.RunAsync(ragFunction, new(query)
{
    ["data"] = augmentedText    
});

Console.WriteLine(result);